In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-03-28 09:33:01.236690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 09:33:01.236767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 09:33:01.239935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 09:33:01.260181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 09:33:03.946853: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../melanoma_cancer_dataset"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/9605
[INFO]: Processed 2000/9605
[INFO]: Processed 3000/9605
[INFO]: Processed 4000/9605
[INFO]: Processed 5000/9605
[INFO]: Processed 6000/9605
[INFO]: Processed 7000/9605
[INFO]: Processed 8000/9605
[INFO]: Processed 9000/9605
(9605, 224, 224, 3)
(9605,)


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-03-28 09:34:10.695180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37473 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-03-28 09:34:21.800863: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-28 09:34:22.468014: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


301/301 [==============================] - 9s 19ms/step
(9605, 25088)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9292403746097815
Sensitivity: 0.9240780911062907
Specificity: 0.934
AUC-ROC: 0.9290390455531453
MCC: 0.8582307617842102
Precision: 0.9281045751633987
F1 Score: 0.9260869565217392

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9136316337148803
Sensitivity: 0.9219088937093276
Specificity: 0.906
AUC-ROC: 0.9139544468546638
MCC: 0.8273563109014886
Precision: 0.9004237288135594
F1 Score: 0.9110396570203646

[INFO] Fold 3 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9157127991675338
Sensitivity: 0.9067245119305857
Specificity: 0.924
AUC-ROC: 0.9153622559652927
MCC: 0.8311212371417839
Precision: 0.9166666666666666
F1 Score: 0.9116684841875682

[INFO] Fold 4 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9053069719042663
Sensitivity: 0.8893709327548807
Specificity: 0.92
AUC-ROC: 0.9046854663774404
MCC: 0.8103382911987241
Precision: 0.9111111111111111
F1 Score: 0.9001097694840834

[INFO] Fold 5 / 10 for LR
Accuracy: 0.8959417273673257
Sensitivity: 0.8655097613882863
Specificity: 0.924
AUC-ROC: 0.8947548806941432
MCC: 0.7921120258827298
Precision: 0.9130434782608695
F1 Score: 0.8886414253897551

[INFO] Fold 6 / 10 for LR
Accuracy: 0.9145833333333333
Sensitivity: 0.9
Specificity: 0.928
AUC-ROC: 0.914
MCC: 0.8289014700546884
Precision: 0.92
F1 Score: 0.90989010989011

[INFO] Fold 7 / 10 for LR
Accuracy: 0.9166666666666666
Sensitivity: 0.9152173913043479
Specificity: 0.918
AUC-ROC: 0.916608695652174
MCC: 0.8330797634074112
Precision: 0.9112554112554112
F1 Score: 0.913232104121475

[INFO] Fold 8 / 10 for LR
Accuracy: 0.9135416666666667
Sensitivity: 0.9065217391304348
Specificity: 0.92
AUC-ROC: 0.9132608695652173
MCC: 0.8267536221213136
Precision: 0.912472647702407
F1 Score: 0.909

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8074921956295525
Sensitivity: 0.702819956616052
Specificity: 0.904
AUC-ROC: 0.8034099783080261
MCC: 0.6223951540804369
Precision: 0.8709677419354839
F1 Score: 0.7779111644657862

[INFO] Fold 2 / 10 for NB
Accuracy: 0.8012486992715921
Sensitivity: 0.7180043383947939
Specificity: 0.878
AUC-ROC: 0.7980021691973971
MCC: 0.6058787478393683
Precision: 0.8443877551020408
F1 Score: 0.7760844079718641

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8043704474505723
Sensitivity: 0.702819956616052
Specificity: 0.898
AUC-ROC: 0.800409978308026
MCC: 0.6153403193612692
Precision: 0.864
F1 Score: 0.7751196172248804

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8012486992715921
Sensitivity: 0.737527114967462
Specificity: 0.86
AUC-ROC: 0.7987635574837311
MCC: 0.6035668167133424
Precision: 0.8292682926829268
F1 Score: 0.7807118254879448

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8012486992715921
Sensitivity: 0.6941431670281996
Specificity: 0.9
AUC-ROC: 0.7970715835140999
MCC: 0.610002

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.9146722164412071
Sensitivity: 0.8655097613882863
Specificity: 0.96
AUC-ROC: 0.9127548806941431
MCC: 0.831669975559326
Precision: 0.9522673031026253
F1 Score: 0.9068181818181817

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.899063475546306
Sensitivity: 0.8633405639913232
Specificity: 0.932
AUC-ROC: 0.8976702819956616
MCC: 0.7987647452769947
Precision: 0.9212962962962963
F1 Score: 0.8913773796192609

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.8896982310093653
Sensitivity: 0.8156182212581344
Specificity: 0.958
AUC-ROC: 0.8868091106290672
MCC: 0.7849235325930612
Precision: 0.947103274559194
F1 Score: 0.8764568764568764

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.8980228928199792
Sensitivity: 0.8438177874186551
Specificity: 0.948
AUC-ROC: 0.8959088937093276
MCC: 0.7986203062979013
Precision: 0.9373493975903614
F1 Score: 0.8881278538812785

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.8896982310093653
Sensitivity: 0.8264642082429501
Specificity: 0.948
AUC-ROC: 0.887232104

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9167533818938606
Sensitivity: 0.8893709327548807
Specificity: 0.942
AUC-ROC: 0.9156854663774403
MCC: 0.8338017252887762
Precision: 0.9339407744874715
F1 Score: 0.9111111111111111

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9240374609781478
Sensitivity: 0.8980477223427332
Specificity: 0.948
AUC-ROC: 0.9230238611713666
MCC: 0.848369654199395
Precision: 0.9409090909090909
F1 Score: 0.9189789123196449

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9240374609781478
Sensitivity: 0.8893709327548807
Specificity: 0.956
AUC-ROC: 0.9226854663774402
MCC: 0.8490105098850389
Precision: 0.9490740740740741
F1 Score: 0.9182530795072787

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9157127991675338
Sensitivity: 0.8741865509761388
Specificity: 0.954
AUC-ROC: 0.9140932754880694
MCC: 0.8328790689071761
Precision: 0.9460093896713615
F1 Score: 0.9086809470124013

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.9105098855359001
Sensitivity: 0.869

In [15]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 843.847393 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3429483
[LightGBM] [Info] Number of data points in the train set: 8644, number of used features: 23825
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.479408 -> initscore=-0.082416
[LightGBM] [Info] Start training from score -0.082416
Accuracy: 0.9354838709677419
Sensitivity: 0.9219088937093276
Specificity: 0.948
AUC-ROC: 0.9349544468546637
MCC: 0.8708350025349626
Precision: 0.9423503325942351
F1 Score: 0.9320175438596491

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 460.409496 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] T

In [16]:
# Save results to a CSV file
results_df.to_csv('DS4_VGG16.csv', index=False)

In [17]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.914317,0.905759,0.9222,0.913980,0.828439,0.914729,0.910138,-532.425781,969.761266
1,NB,0.800311,0.705314,0.8878,0.796557,0.605978,0.853084,0.771948,827.867188,126.095018
2,KNN,0.898074,0.843432,0.9484,0.895916,0.798903,0.937858,0.888023,797.394531,180.175986
3,Random Forest,0.919730,0.888169,0.9488,0.918485,0.840147,0.941163,0.913840,858.039062,223.414469
4,LGBM,0.927537,0.907055,0.9464,0.926727,0.855225,0.939841,0.923084,1284.429688,19227.640634
